## Text cleaning and spell checking

This notebook contains beginning efforts for spell checking. Formatting (conversion to integers etc.) has been moved to scraping methods.

In [2]:
import pickle, string, re

In [4]:
from enchant.checker import SpellChecker
from enchant.tokenize import EmailFilter, URLFilter
import enchant

import nltk

from collections import Counter

In [5]:
from drug_objects import drug, review

In [6]:
from cleaning_refs import med_dur_conversion as med_dur_conversion
from cleaning_refs import cList as cList
new_words = open("new_words.txt", "rb").readlines()
from cleaning_refs import new_words as new_words

In [7]:
def load_pickle(_pickle_name):
    return pickle.load( open( _pickle_name, "rb" ) )

In [8]:
inverse_cList = {}
for word in cList:
    new_key = word.replace("'", "")
    inverse_cList[new_key] = cList[word]

cList_keys = cList.keys()
inverse_cList_keys = inverse_cList.keys()

In [9]:
def expandContractions(text, words):#, c_re=c_re):
    c_re = re.compile('(%s)' % '|'.join(words.keys()))
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [21]:
#method of review?
def metaData_clean(_reviews):
    for ik, _review in enumerate(_reviews):
        #pull out contractions
        comment_text = (_review.comment).split()
        for ik, word in enumerate(comment_text):
            if len(word.split("'s"))>1:
                comment_text[ik] = word.split("'s")[0]
            if word in cList_keys:
                comment_text[ik]= cList[word]
            if word in inverse_cList_keys:
                comment_text[ik] = inverse_cList[word]

        _review.comment = ' '.join(comment_text)
        
        #remove punctuation
        remove = string.punctuation
        remove = remove.replace("-", "") # don't remove hyphens
        pattern = r"[{}]".format(remove) 
        _review.comment = re.sub(pattern, "",  _review.comment) 

    return _reviews

In [36]:
d2 = enchant.DictWithPWL("en_US","new_words.txt")
chkr = SpellChecker("en_US", filters=[EmailFilter,URLFilter])

def spellchecker(reviews):
    for ik, review in enumerate(reviews):
            chkr.set_text(review.comment)
            for err in chkr:
                # forFuture: add catch for repeated character, e.g. 'soooooooo'
                error = (err.word).lower()
                options = [change.lower() for change in d2.suggest(error)]
                if error in options:
                    continue
                else:
                    catch = 'no'
                    word_to_beat = options[0]
                    score_to_beat = 10
                    for word in options:
                        dist = nltk.edit_distance(error, word)
                        if (word in new_words):
                            if (nltk.edit_distance(error, word)<=score_to_beat):
                                word_to_beat = word
                                score_to_beat = nltk.edit_distance(error, word)
                                catch = 'yes'
                        elif dist<score_to_beat:
                            word_to_beat = word
                            score_to_beat = nltk.edit_distance(error, word)
                    print(word_to_beat)
                    print('final option:', word_to_beat , ', error:', error, 'score:', score_to_beat)

In [12]:
pickle_name = "drug_list_ddc_pg1.p"

In [13]:
drug_list = load_pickle(pickle_name)
print(len(drug_list[0].reviews))

450


In [22]:
drug_list[0].reviews = metaData_clean(drug_list[0].reviews)

In [37]:
spellchecker(drug_list[0].reviews)

cymbalta
 , error: cymbalta score: 1
lamictal
 , error: lamictal score: 1
eds
final option: eds , error: meds score: 1
cymbalta
 , error: cymbalta score: 1
cymbalta
 , error: cymbalta score: 1
cymbalta
 , error: cymbalta score: 1
ptsd
 , error: ptsd score: 1
cymbalta
 , error: cymbalta score: 1
realized
final option: realized , error: realised score: 1
sertraline
 , error: sertraline score: 1
duloxetine
 , error: duloxetine score: 1
cymbalta
 , error: cymbalta score: 1
cymbalta
 , error: cymbalta score: 1
cymbalta
 , error: cymbalta score: 1
prissy
final option: prissy , error: pissy score: 1
cu
final option: cu , error: cuz score: 1
zoloft
 , error: zoloft score: 1
zoloft
 , error: zoloft score: 1
zoloft
 , error: zoloft score: 1
kingliness
final option: kingliness , error: tingliness score: 1
eds
final option: eds , error: meds score: 1
cymbalta
 , error: cymbalta score: 1
saris
final option: saris , error: ssris score: 1
cymbalta
 , error: cymbalta score: 1
cymbalta
 , error: cymbal

IndexError: list index out of range